### 1. Download Shopify dataset and create table script:

In [1]:
!gsutil cp gs://cs327e-open-access/spanner_shopify.zip /home/jupyter/snippets

Copying gs://cs327e-open-access/spanner_shopify.zip...
/ [1 files][  3.5 MiB/  3.5 MiB]                                                
Operation completed over 1 objects/3.5 MiB.                                      


In [4]:
!unzip /home/jupyter/snippets/spanner_shopify.zip 

Archive:  /home/jupyter/snippets/spanner_shopify.zip
replace spanner_shopify/pricing_plan_features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


### 2. Configure Spanner CLI and create Shopify Database

In [1]:
%env CLI=/home/jupyter/spanner-cli-0.9.9/spanner-cli
%env SPAN_PROJECT=vocal-tracker-324800
%env SPAN_INSTANCE=span
%env SPAN_DATABASE=span

env: CLI=/home/jupyter/spanner-cli-0.9.9/spanner-cli
env: SPAN_PROJECT=vocal-tracker-324800
env: SPAN_INSTANCE=span
env: SPAN_DATABASE=span


In [2]:
!$CLI -p $SPAN_PROJECT -i $SPAN_INSTANCE -d $SPAN_DATABASE -e "show tables"

Tables_in_span
apps
apps_categories
categories
key_benefits
pricing_plan_features
pricing_plans


In [3]:
CLI_CONNECT="$CLI -p $SPAN_PROJECT -i $SPAN_INSTANCE -d $SPAN_DATABASE"

In [4]:
!{CLI_CONNECT} -e "create database shopify"

ERROR: rpc error: code = AlreadyExists desc = Database already exists: projects/vocal-tracker-324800/instances/span/databases/shopify
error details: name = ResourceInfo type = type.googleapis.com/google.spanner.admin.database.v1.Database resourcename = projects/vocal-tracker-324800/instances/span/databases/shopify owner =  desc = Database already exists.


In [5]:
%env SPAN_DATABASE=shopify

env: SPAN_DATABASE=shopify


In [6]:
!$CLI -p $SPAN_PROJECT -i $SPAN_INSTANCE -d $SPAN_DATABASE -e "show tables"

Tables_in_shopify
apps
apps_categories
categories
key_benefits
pricing_plan_features
pricing_plans
reviews


In [7]:
!{CLI_CONNECT} -f /home/jupyter/snippets/spanner_shopify/create_tables.sql

ERROR: rpc error: code = FailedPrecondition desc = Duplicate name in schema: apps.


In [8]:
!{CLI_CONNECT} -e "show tables"

Tables_in_shopify
apps
apps_categories
categories
key_benefits
pricing_plan_features
pricing_plans
reviews


In [9]:
!{CLI_CONNECT} -e "show create table key_benefits"

Table	Create Table
key_benefits	CREATE TABLE key_benefits (
  app_id STRING(50),
  title STRING(2000),
  description STRING(MAX),
) PRIMARY KEY(app_id, title)


In [12]:
!{CLI_CONNECT} -e "show create table apps"

Table	Create Table
apps	CREATE TABLE apps (
  id STRING(50),
  url STRING(1000),
  title STRING(2000),
  developer STRING(100),
  developer_link STRING(1000),
  icon STRING(1000),
  rating FLOAT64,
  reviews_count INT64,
) PRIMARY KEY(id)


In [39]:
!{CLI_CONNECT} -e "show create table pricing_plans"

Table	Create Table
pricing_plans	CREATE TABLE pricing_plans (
  id STRING(50),
  app_id STRING(50),
  title STRING(200),
  price FLOAT64,
) PRIMARY KEY(id)


In [14]:
!{CLI_CONNECT} -e "show create table apps_categories"

Table	Create Table
apps_categories	CREATE TABLE apps_categories (
  app_id STRING(50),
  category_id STRING(50),
) PRIMARY KEY(app_id, category_id)


In [15]:
!{CLI_CONNECT} -e "show create table pricing_plan_features"

Table	Create Table
pricing_plan_features	CREATE TABLE pricing_plan_features (
  pricing_plan_id STRING(50),
  app_id STRING(50),
  feature STRING(MAX),
) PRIMARY KEY(pricing_plan_id, app_id, feature)


In [16]:
!{CLI_CONNECT} -e "show create table categories"

Table	Create Table
categories	CREATE TABLE categories (
  id STRING(50),
  title STRING(2000),
) PRIMARY KEY(id)


In [17]:
!{CLI_CONNECT} -e "show create table reviews"

Table	Create Table
reviews	CREATE TABLE reviews (
  app_id STRING(50),
  author STRING(250),
  rating INT64,
  posted_at DATE,
) PRIMARY KEY(app_id, author)


### 3. Populate tables

In [10]:
from google.cloud import spanner
from google.cloud.spanner_v1 import param_types
import math, pandas as pd

In [11]:
span_instance='span'
span_database='shopify'
dir_path = '/home/jupyter/snippets/spanner_shopify'
spanner_client = spanner.Client()
instance = spanner_client.instance(span_instance)
database = instance.database(span_database)

In [12]:
def split_dataframe(df, split_size): 
    splits = list()
    num_splits = math.ceil(len(df) / split_size) 
    for i in range(0, len(df), split_size):
        df_subset = df.iloc[i:i + split_size]
        splits.append(df_subset)
    return splits

In [31]:
table = 'apps'
df = pd.read_csv(dir_path + '/apps.csv', sep=',', header=0, lineterminator='\n')

df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('id', 'url', 'title', 'developer', 'developer_link', 'icon', 'rating', 'reviews_count'),
            values=rows
        )

inserting 1000 rows into apps


AlreadyExists: 409 Row [00312cea-3595-4d65-a60d-6e8e5b7d6897] in table apps already exists

In [8]:
table = 'apps_categories'
df = pd.read_csv(dir_path + '/apps_categories.csv', sep=',', header=0, lineterminator='\n')
df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)
    
    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('app_id', 'category_id'),
            values=rows
        )

inserting 1000 rows into apps_categories


AlreadyExists: 409 Row [00289a9f-9f12-45b1-963b-67e78403f7c7,5eb4e29e50e3f178acc614236ed107f4] in table apps_categories already exists

In [28]:
table = 'categories'
df = pd.read_csv(dir_path + '/categories.csv', sep=',', header=0, lineterminator='\n')
rows = df.values.tolist()
print('inserting ' + str(len(rows)) + ' rows into ' + table)
    
with database.batch() as batch:
    batch.insert(
        table=table,
        columns=('id', 'title'),
        values=rows
    )

inserting 12 rows into categories


AlreadyExists: 409 Row [26a72de0d02e0e4e5f615332d61a878e] in table categories already exists

In [ ]:
table = 'key_benefits'
df = pd.read_csv(dir_path + '/key_benefits.csv', sep=',', header=0, lineterminator='\n')
df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('app_id', 'title', 'description'),
            values=rows
        )

In [ ]:
table = 'pricing_plans'
df = pd.read_csv(dir_path + '/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('id', 'app_id', 'title', 'price'),
            values=rows
        )

In [29]:
table = 'pricing_plan_features'
df = pd.read_csv(dir_path + '/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('pricing_plan_id', 'app_id', 'feature'),
            values=rows
        )

inserting 1000 rows into pricing_plan_features


AlreadyExists: 409 Row [00decc4c-91d2-4616-b696-f53bc29d2937,0579c9d7-d43c-4210-855e-20626b23816a,Basic button template] in table pricing_plan_features already exists

In [ ]:
!{CLI_CONNECT} -e "select count(*) from apps"

In [ ]:
!{CLI_CONNECT} -e "select count(*) from apps_categories"

In [9]:
!{CLI_CONNECT} -e "select count(*) from categories"

/bin/bash: {CLI_CONNECT}: command not found


In [34]:
!{CLI_CONNECT} -e "select count(*) from key_benefits"


9541


In [20]:
!{CLI_CONNECT} -e "select count(*) from pricing_plans"


6275


In [32]:
!{CLI_CONNECT} -e "select count(*) from pricing_plan_features"


16270


In [41]:
def insert_app(transaction):

    row_ct = transaction.execute_update(
        "INSERT INTO apps (id, url, title, developer, developer_link, icon, reviews_count) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', "
          "'https://apps.shopify.com/watchlist?surface_detail=inventory-management&surface_inter_position=1&surface_intra_position=9&surface_type=category'," 
          "'Back in Stock Product Alerts', "
          "'Swym Corporation', "
          "'https://apps.shopify.com/partners/developer-ca6a967f09890f68',"
          "'https://apps.shopifycdn.com/listing_images/9905a4c8f22cb4a3b0c32af55a58ec21/icon/e6d46a7e5e1df375d542d033aae80459.png?height=72&width=72',"
           "0)"
    )
    
    print("{} apps record(s) inserted.".format(row_ct))
    
    row_ct = transaction.execute_update(
        "INSERT INTO apps_categories (app_id, category_id) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', '737ad50051083aa051d127a53b3ac0da')"
    )
    
    print("{} apps_categories record(s) inserted.".format(row_ct))
    
    
    row_ct = transaction.execute_update(
        "INSERT INTO key_benefits (app_id, title, description) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', "
          "'Drive Back In Stock Sales', 'Back in stock, out of stock, pre order & restock alerts')"
    )
    
    print("{} key_benefits record(s) inserted.".format(row_ct))
    
    row_ct = transaction.execute_update(
        "INSERT INTO pricing_plans (id, app_id, title, price) "
        "VALUES ('961a2751-1dbd-4f0f-8248-9fe6a6efd4ab', '683d06af-14c7-4733-9bde-ec5b699af996', "
                "'Free Trial', 14.99)"
    )
    
    print("{} pricing_plans record(s) inserted.".format(row_ct))
    

    row_ct = transaction.execute_update(
        "INSERT INTO pricing_plan_features (app_id, pricing_plan_id, feature) "
        " VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', '961a2751-1dbd-4f0f-8248-9fe6a6efd4ab', "
                 "'Up to 250 alert requests/mo')"
    )
    
    print("{} pricing_plan_features record(s) inserted.".format(row_ct))
      
database.run_in_transaction(insert_app)

AlreadyExists: 409 Row [683d06af-14c7-4733-9bde-ec5b699af996] in table apps already exists

In [42]:
def update_price(transaction):
    row_ct = transaction.execute_update(
            "UPDATE pricing_plans "
            "SET price = price / 2 "
            "WHERE price > 0"
        )

    print("{} pricing_plans record(s) updated.".format(row_ct))

database.run_in_transaction(update_price)

4780 pricing_plans record(s) updated.


In [43]:
def delete_app(transaction):
        row_ct = transaction.execute_update(
            "DELETE FROM apps WHERE id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )

        print("{} apps record(s) deleted.".format(row_ct))
        
        row_ct = transaction.execute_update(
            "DELETE FROM apps_categories WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} apps_categories record(s) deleted.".format(row_ct))
        
        row_ct = transaction.execute_update(
            "DELETE FROM key_benefits WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} key_benefits record(s) deleted.".format(row_ct)) 
        
        
        row_ct = transaction.execute_update(
            "DELETE FROM pricing_plans WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} pricing_plans record(s) deleted.".format(row_ct)) 
        
        
        row_ct = transaction.execute_update(
            "DELETE FROM pricing_plan_features WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} pricing_plan_features record(s) deleted.".format(row_ct))

database.run_in_transaction(delete_app)

1 apps record(s) deleted.
1 apps_categories record(s) deleted.
1 key_benefits record(s) deleted.
1 pricing_plans record(s) deleted.
1 pricing_plan_features record(s) deleted.


In [48]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories \
  ADD CONSTRAINT apps_cat_app_id_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

ERROR: rpc error: code = FailedPrecondition desc = Duplicate name in schema: apps_cat_app_id_fk.


In [40]:
!{CLI_CONNECT} -e "show create table apps_categories"

Table	Create Table
apps_categories	CREATE TABLE apps_categories (
  app_id STRING(50),
  category_id STRING(50),
) PRIMARY KEY(app_id, category_id)


In [39]:
!{CLI_CONNECT} -e "CREATE INDEX apps_rating_idx ON apps(rating)"

In [12]:
!{CLI_CONNECT} -e "CREATE INDEX app_reviews_count_idx ON apps(reviews_count)"

/bin/bash: {CLI_CONNECT}: command not found


## begin project 3 work

In [13]:
table = 'reviews'
df = pd.read_csv(dir_path + '/reviews.csv', sep=',', header=0, lineterminator='\n')
df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=("app_id", "author","rating","posted_at"),
            values=rows
        )
        

inserting 1000 rows into reviews


AlreadyExists: 409 Row [00289a9f-9f12-45b1-963b-67e78403f7c7,Hellobeanies] in table reviews already exists

In [17]:
!{CLI_CONNECT} -e "select count(*) from reviews"


224601


In [36]:
!{CLI_CONNECT} -e "show create table reviews"

ERROR: table "reviews" doesn't exist


## Insert 100,000 fake reviews into the reviews table. 
## Reviews should be inserted in batches of 1000 using the batch_insert method

In [16]:
import numpy as np
import random

table = 'reviews'
l = []
for i in range(100000):
    random_name = "Homer Simpson"+ str(i)
    l.append(["6286d4cd-3fcb-4ee3-bb3c-de051c28b83c",random_name,random.randint(1,5),"2021-09-20"])
data = np.asarray(l)
fakedf = pd.DataFrame(data,columns = ["app_id", "author","rating","posted_at"])

df_splits = split_dataframe(fakedf, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=("app_id", "author","rating","posted_at"),
            values=rows
        )

inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 

In [21]:
!{CLI_CONNECT} -e "select AVG(rating) from reviews where app_id = '6286d4cd-3fcb-4ee3-bb3c-de051c28b83c'"


2.994470


## Update the corresponding apps record in the apps table with an average rating and 
## review count based on your assigned values.

In [31]:
rating1 = !{CLI_CONNECT} -e "select AVG(rating) from reviews where app_id = '6286d4cd-3fcb-4ee3-bb3c-de051c28b83c'"
truerating = float(rating1[1])
print(truerating)
def update_rating(transaction):
    row_ct = transaction.execute_update(
            "UPDATE apps "
            f"SET rating = {truerating} "
            "WHERE id = '6286d4cd-3fcb-4ee3-bb3c-de051c28b83c'"
        )

    print("{} rating record(s) updated.".format(row_ct))

    
database.run_in_transaction(update_rating)    


2.99447
1 rating record(s) updated.


In [34]:
count1 = !{CLI_CONNECT} -e "select count(*) from reviews where app_id = '6286d4cd-3fcb-4ee3-bb3c-de051c28b83c' "
print(count1)

['', '100000']


In [38]:
count1 = !{CLI_CONNECT} -e "select count(*) from reviews where app_id = '6286d4cd-3fcb-4ee3-bb3c-de051c28b83c' "
truecount = int(count1[1])
def update_reviews_count(transaction):
    row_ct = transaction.execute_update(
            "UPDATE apps "
            f"SET reviews_count = {truecount} "
            "WHERE id = '6286d4cd-3fcb-4ee3-bb3c-de051c28b83c'"
        )

    print("{} review_count record(s) updated.".format(row_ct))



database.run_in_transaction(update_reviews_count)

1 review_count record(s) updated.


In [40]:
!{CLI_CONNECT} -e "Create INDEX author_idx ON reviews(author)"

## We index columns included in the 'where' clause so that it would be easier and faster to look up those conditions. We indexed based on author in the reviews table and used the index we made in class for the app table. We saw a small increase in the execution time after we used the indexes.

In [44]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories \
  ADD CONSTRAINT category_fk FOREIGN KEY (category_id) REFERENCES categories (id)"

In [46]:
!{CLI_CONNECT} -e "ALTER TABLE key_benefits \
  ADD CONSTRAINT get_app_id_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

In [48]:
!{CLI_CONNECT} -e "ALTER TABLE pricing_plan_features \
  ADD CONSTRAINT get_app_id_ppf_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

In [49]:
!{CLI_CONNECT} -e "ALTER TABLE pricing_plan_features \
  ADD CONSTRAINT get_prcing_id_ppf_fk FOREIGN KEY (pricing_plan_id) REFERENCES pricing_plans (id)"

In [50]:
!{CLI_CONNECT} -e "ALTER TABLE reviews \
  ADD CONSTRAINT get_app_id_r_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

## We modified 4 different tables; apps_categories, key_benefits, pricing_plan_features, and reviews. In apps_categories we added the foreign key category_id from categories. In key_benefits we added the foreign key app_id from app. In pricing_plan_features we added 2 foreign keys, app_id from apps and pricing_plan_id from pricing_plans. In reviews we added the foreign key app_id from apps.

In [52]:
!{CLI_CONNECT} -e "show create table apps_categories"

Table	Create Table
apps_categories	CREATE TABLE apps_categories (
  app_id STRING(50),
  category_id STRING(50),
  CONSTRAINT apps_cat_app_id_fk FOREIGN KEY(app_id) REFERENCES apps(id),
) PRIMARY KEY(app_id, category_id)


In [53]:
!{CLI_CONNECT} -e "show create table key_benefits"

Table	Create Table
key_benefits	CREATE TABLE key_benefits (
  app_id STRING(50),
  title STRING(2000),
  description STRING(MAX),
  CONSTRAINT get_app_id FOREIGN KEY(app_id) REFERENCES apps(id),
  CONSTRAINT get_app_id_fk FOREIGN KEY(app_id) REFERENCES apps(id),
) PRIMARY KEY(app_id, title)


In [54]:
!{CLI_CONNECT} -e "show create table pricing_plan_features"

Table	Create Table
pricing_plan_features	CREATE TABLE pricing_plan_features (
  pricing_plan_id STRING(50),
  app_id STRING(50),
  feature STRING(MAX),
  CONSTRAINT get_app_id_ppf_fk FOREIGN KEY(app_id) REFERENCES apps(id),
) PRIMARY KEY(pricing_plan_id, app_id, feature)


In [55]:
!{CLI_CONNECT} -e "show create table reviews"

Table	Create Table
reviews	CREATE TABLE reviews (
  app_id STRING(50),
  author STRING(250),
  rating INT64,
  posted_at DATE,
  CONSTRAINT get_app_id_r_fk FOREIGN KEY(app_id) REFERENCES apps(id),
) PRIMARY KEY(app_id, author)
